In [1]:
!pip install beautifulsoup4 requests

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import random

# Путь к CSV файлу
file_path = '/content/links.csv'
data = pd.read_csv(file_path)

# Диапазон строк для обработки
start_row = 0
end_row = len(data)

# Заголовки для имитации браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Проверка на наличие украинских символов
def contains_ukrainian(text):
    return re.search(r'[ҐґЄєІіЇї]', text)

# Функция для получения содержимого страницы
def get_page_content(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе {url}: {e}")
        return None

# Функция для парсинга данных
def extract_metadata(soup):
    metadata = {}

    # Title
    title = soup.find('meta', {'name': 'citation_title'})
    if title:
        metadata['Title'] = title['content']
        # Проверка на украинские символы в заголовке
        if contains_ukrainian(metadata['Title']):
            return None  # Игнорируем статьи с украинскими символами в заголовке

        # Проверка, что в заголовке есть хотя бы одно русское слово (наличие кириллических символов)
        if not re.search(r'[А-Яа-я]', metadata['Title']):
            return None  # Если в заголовке нет русских букв, игнорируем статью
    else:
        return None  # Если нет заголовка, игнорируем статью

    # Authors
    authors = soup.find_all('meta', {'name': 'citation_author'})
    if authors:
        author_list = [author['content'] for author in authors]
        metadata['Authors'] = ', '.join(author_list)
    else:
        metadata['Authors'] = ""

    # Journal
    journal = soup.find('meta', {'name': 'citation_journal_title'})
    metadata['Journal'] = journal['content'] if journal else ""

    # Year
    year = soup.find('meta', {'name': 'citation_publication_date'})
    metadata['Year'] = year['content'] if year else ""

    # Field of Science
    field_of_science_block = soup.find('div', class_='half-right')
    if field_of_science_block:
        field_of_science = field_of_science_block.find('a')
        metadata['Field of Science'] = field_of_science.text if field_of_science else ""
    else:
        metadata['Field of Science'] = ""

    # Russian and English Keywords extraction with correct handling
    keywords = soup.find('meta', {'name': 'citation_keywords'})
    if keywords:
        keyword_list = [kw.strip().lower() for kw in keywords['content'].split(',')]

        russian_keywords = []
        english_keywords = []

        for kw in keyword_list:
            if contains_ukrainian(kw):
                return None  # Игнорируем статьи с украинскими символами в ключевых словах

            # Слово содержит кириллические символы или это смесь русского и английского
            if re.search(r'[А-Яа-я]', kw):
                russian_keywords.append(kw)
            else:
                english_keywords.append(kw)

        metadata['Russian Keywords'] = ', '.join(russian_keywords) if russian_keywords else ""
        metadata['English Keywords'] = ', '.join(english_keywords) if english_keywords else ""
    else:
        metadata['Russian Keywords'] = ""
        metadata['English Keywords'] = ""

    # Annotation
    annotation = soup.find('p', itemprop='description')
    metadata['Annotation'] = annotation.text.strip() if annotation else ""

    # Извлечение данных из JavaScript
    script_tag = soup.find('script', string=re.compile('gost_electronic'))
    if script_tag:
        script_content = script_tag.string

        # GOST citation
        gost_match = re.search(r'gost_electronic: "(.*?)"', script_content)
        metadata['GOST'] = gost_match.group(1).replace('\/', '/').replace('\\u0026', '&') if gost_match else ""

        # APA citation
        apa_match = re.search(r'apa: "(.*?)"', script_content)
        metadata['APA'] = apa_match.group(1).replace('\/', '/').replace('\\u0026', '&') if apa_match else ""
    else:
        metadata['GOST'] = ""
        metadata['APA'] = ""

    # Link
    link = soup.find('meta', {'name': 'citation_fulltext_html_url'})
    metadata['Link'] = link['content'] if link else ""

    return metadata

# Проход по каждой строке в указанном диапазоне
results = []
for index, row in data.iloc[start_row:end_row].iterrows():
    url = row['Link']  # Замените на правильное имя столбца
    print(f"Обработка ссылки: {url}")

    page_content = get_page_content(url)
    if page_content:
        soup = BeautifulSoup(page_content, 'html.parser')
        metadata = extract_metadata(soup)
        if metadata:
            results.append(metadata)

    # Задержка между запросами
    time.sleep(random.uniform(1, 3))  # Случайная задержка от 1 до 3 секунд

# Преобразование результатов в DataFrame и добавление в CSV
result_df = pd.DataFrame(results)
output_file = 'carbon_polygon.csv'
result_df.to_csv(output_file, mode='a', header=not pd.io.common.file_exists(output_file), index=False)


Обработка ссылки: https://cyberleninka.ru/article/n/rynok-tehnologii-i-metody-otsenki-effektivnosti-sovremennyh-mnogofunktsionalnyh-prisadok-k-dizelnomu-toplivu
Обработка ссылки: https://cyberleninka.ru/article/n/mesta-defosfatirovaniya-gorodskoy-stochnoy-zhidkosti-i-effekt-udaleniya-fosfora-reagentami
Обработка ссылки: https://cyberleninka.ru/article/n/gidrofobizatsiya-tonkodispersnogo-mela
Обработка ссылки: https://cyberleninka.ru/article/n/tehniko-ekonomicheskiy-analiz-vysokotemperaturnoy-pererabotki-kislyh-gudronov-v-rasplavlennoy-srede
Обработка ссылки: https://cyberleninka.ru/article/n/ispolzovanie-karbonovyh-ferm-dlya-fiksatsii-uglekislogo-gaza
Обработка ссылки: https://cyberleninka.ru/article/n/k-voprosu-o-pravovom-regulirovanii-rynka-uglerodnyh-edinits-sravnitelno-pravovoy-aspekt
Обработка ссылки: https://cyberleninka.ru/article/n/pravovoe-regulirovanie-mehanizma-sderzhivaniya-izmeneniya-klimata-v-lesnom-hozyaystve
Обработка ссылки: https://cyberleninka.ru/article/n/pravovye-p

In [ ]:
import csv

# Список всех ссылок
links = [
    # Химические технологии (5)
    "https://cyberleninka.ru/article/n/rynok-tehnologii-i-metody-otsenki-effektivnosti-sovremennyh-mnogofunktsionalnyh-prisadok-k-dizelnomu-toplivu",
    "https://cyberleninka.ru/article/n/mesta-defosfatirovaniya-gorodskoy-stochnoy-zhidkosti-i-effekt-udaleniya-fosfora-reagentami",
    "https://cyberleninka.ru/article/n/gidrofobizatsiya-tonkodispersnogo-mela",
    "https://cyberleninka.ru/article/n/tehniko-ekonomicheskiy-analiz-vysokotemperaturnoy-pererabotki-kislyh-gudronov-v-rasplavlennoy-srede",
    "https://cyberleninka.ru/article/n/ispolzovanie-karbonovyh-ferm-dlya-fiksatsii-uglekislogo-gaza",

    # Право (9)
    "https://cyberleninka.ru/article/n/k-voprosu-o-pravovom-regulirovanii-rynka-uglerodnyh-edinits-sravnitelno-pravovoy-aspekt",
    "https://cyberleninka.ru/article/n/pravovoe-regulirovanie-mehanizma-sderzhivaniya-izmeneniya-klimata-v-lesnom-hozyaystve",
    "https://cyberleninka.ru/article/n/pravovye-pozitsii-verhovnogo-suda",
    "https://cyberleninka.ru/article/n/vozmozhnosti-razvitiya-munitsipalitetov-s-uchetom-novoy-esg-strategii",
    "https://cyberleninka.ru/article/n/pravovye-problemy-prirodopolzovaniya-i-ohrany-okruzhayuschey-sredy-pri-nizkouglerodnyh-edinitsah",
    "https://cyberleninka.ru/article/n/sootnoshenie-ponyatiy-yazyk-ugolovnogo-sudoproizvodstva-i-yazyk-kriminalistiki",
    "https://cyberleninka.ru/article/n/sahalinskiy-eksperiment-po-snizheniyu-parnikovyh-vybrosov",
    "https://cyberleninka.ru/article/n/pravovoe-regulirovanie-dekarbonizatsii-i-ohrany-atmosfernogo-vozduha-v-tselyah-resheniya-globalnoy-ekologicheskoy-problemy",
    "https://cyberleninka.ru/article/n/ekologo-pravovoe-obespechenie-realizatsii-pilotnogo-proekta-po-sozdaniyu-karbonovyh-poligonov-i-karbonovyh-ferm",

    # Экологические биотехнологии (3)
    "https://cyberleninka.ru/article/n/potentsial-ispolzovaniya-vodyanogo-oreha-trapa-natans-dlya-ochistki-vodoemov-ot-neftyanogo-zagryazneniya",
    "https://cyberleninka.ru/article/n/sravnitelnaya-otsenka-metodov-opredeleniya-dyhaniya-pochvy-vozmozhnosti-ih-ispolzovaniya-v-klimaticheskih-proektah",
    "https://cyberleninka.ru/article/n/metodologicheskie-podhody-formirovaniya-edinoy-natsionalnoy-sistemy-monitoringa-i-ucheta-balansa-ugleroda-i-vybrosov-parnikovyh",

    # Промышленные биотехнологии (5)
    "https://cyberleninka.ru/article/n/k-voprosu-ob-eko-sedobnoy-i-bystrorazlagayuscheysya-upakovke-v-pischevoy-industrii",
    "https://cyberleninka.ru/article/n/novoe-v-mezhdunarodnoy-politike-snizheniya-emissii-parnikovyh-gazov-i-neobhodimye-meropriyatiya-v-morskom-sektore-chast-3",
    "https://cyberleninka.ru/article/n/poluchenie-organomineralnogo-udobreniya-dlya-lesnogo-hozyaystva",
    "https://cyberleninka.ru/article/n/perspektivy-vtorichnogo-ispolzovaniya-i-pererabotki-tekstilnyh-othodov-v-respublike-bashkortostan",
    "https://cyberleninka.ru/article/n/karbonovye-poligony-perspektivy-vnedreniya-v-rf-i-v-respublike-bashkortostan",

    # Социальныя и экономическия география (14)
    "https://cyberleninka.ru/article/n/ob-effektivnosti-ispolzovaniya-prirodnyh-resursov-dlya-obespecheniya-ustoychivogo-razvitiya-territoriy-granichaschih-s-knr",
    "https://cyberleninka.ru/article/n/rol-zelenoy-infrastruktury-v-ustoychivom-razvitii-gorodov",
    "https://cyberleninka.ru/article/n/pogloschenie-ugleroda-lesami-regionov-povolzhya-i-sibiri-sostoyanie-i-perspektivy",
    "https://cyberleninka.ru/article/n/osobennosti-ustoychivogo-razvitiya-regionov-v-usloviyah-nizkouglerodnoy-i-esg-transformatsiy-ekonomiki",
    "https://cyberleninka.ru/article/n/kuzbass-economy-and-carbon-control-tools",
    "https://cyberleninka.ru/article/n/vklad-lesnogo-sektora-v-razvitie-zelenoy-ekonomiki-regiona",
    "https://cyberleninka.ru/article/n/otsenka-perspektiv-dekarbonizatsii-otrasli-rastenievodstva-kabardino-balkarskoy-respubliki",
    "https://cyberleninka.ru/article/n/issledovaniya-ikiba-v-sfere-adaptatsii-sotsio-prirodnyh-kompleksov-regionov-bolshogo-altaya-k-izmeneniyam-klimata",
    "https://cyberleninka.ru/article/n/problemy-transformatsii-lesnogo-hozyaystva-v-selskih-territoriyah-bashkirii",
    "https://cyberleninka.ru/article/n/proektnoe-upravlenie-v-sisteme-ustoychivogo-razvitiya-territoriy-na-primere-michurinskogo-munitsipalnogo-obrazovaniya",
    "https://cyberleninka.ru/article/n/karbonovye-vybrosy-i-monitoring-zagryazneniya-atmosfernogo-vozduha-v-rossiyskoy-federatsii-i-respublike-bashkortostan",
    "https://cyberleninka.ru/article/n/transportnaya-set-respubliki-bashkortostan-kak-osnova-dlya-sozdaniya-karbonovyh-poligonov-s-tselyu-proizvodstva-iskusstvennyh",
    "https://cyberleninka.ru/article/n/dekarbonizatsiya-ekonomiki-analiz-problematiki-s-uchetom-regionalnoy-spetsifiki-i-neobhodimosti-razvitiya-sistemy-obrazovaniya",
    "https://cyberleninka.ru/article/n/sotsialno-ekonomicheskoe-vliyanie-razvitiya-karbonovyh-ferm-na-territorii-regionov",

    # Энергетика и рациональное природопользование (16)
    "https://cyberleninka.ru/article/n/obosnovanie-novyh-podhodov-k-resheniyu-problemy-zagryazneniya-poverhnostnyh-vod-neftedobyvayuschego-regiona-na-primere-respubliki",
    "https://cyberleninka.ru/article/n/ekologicheskie-problemy-utilizatsii-othodov-na-shlamonakopitele-beloe-more-zavoda-kaprolaktam-oao-sibur-neftehim-g-dzerzhinsk",
    "https://cyberleninka.ru/article/n/puti-zaschity-okruzhayuschey-sredy-ot-neftesoderzhaschih-othodov",
    "https://cyberleninka.ru/article/n/eksperimentalnye-issledovaniya-po-tusheniyu-goryuchih-polyarnyh-zhidkostey-kontseptsiya-ognevyh-ispytaniy",
    "https://cyberleninka.ru/article/n/sovremennye-sposoby-obezvrezhivaniya-i-utilizatsii-neftesoderzhaschih-othodov-dlya-likvidatsii-zagryazneniya-okruzhayuschey-sredy",
    "https://cyberleninka.ru/article/n/o-pokazatele-rasshireniya-produktov-vzryva",
    "https://cyberleninka.ru/article/n/neuchtyonnaya-himicheskaya-opasnost-protsessov-transformatsii-veschestv-v-okruzhayuschey-srede-pri-otsenke-effektivnosti-primeneniya",
    "https://cyberleninka.ru/article/n/fiziko-himicheskie-issledovaniya-dlya-otsenki-himicheskoy-bezopasnosti-i-effektivnosti-primeneniya-novoy-sistemy-ochistki-svalochnogo",
    "https://cyberleninka.ru/article/n/puti-dostizheniya-uglerodnoy-neytralnosti-v-rossiyskoy-elektroenergetike",
    "https://cyberleninka.ru/article/n/otsenka-izmeneniya-balansa-parnikovyh-gazov-v-atmosfere-na-osnove-kompleksnoy-geoinformatsionnoy-prostranstvenno-raspredelennoy",
    "https://cyberleninka.ru/article/n/arkticheskiy-vektor-ugolnoy-politiki-v-kontekste-prostranstvennogo-razvitiya-pribrezhnyh-territoriy",
    "https://cyberleninka.ru/article/n/metody-monitoringa-parnikovyh-gazov",
    "https://cyberleninka.ru/article/n/globalnyy-klimat-i-pochvennyy-pokrov-posledstviya-dlya-zemlepolzovaniya-rossii",
    "https://cyberleninka.ru/article/n/uglevodorody-i-obschestvo-problemy-vzaimodeystviya-i-ekologicheskoy-optimizatsii-v-usloviyah-globalnogo-potepleniya-na-nachalo-2022",
    "https://cyberleninka.ru/article/n/izmenenie-klimata-i-uglerodnaya-neytralnost-sovremennye-vyzovy-pered-agrarnoy-naukoy",
    "https://cyberleninka.ru/article/n/karbonovye-poligony-v-respublike-bashkortostan",

    # Биологические науки (19)
     "https://cyberleninka.ru/article/n/biosfernyy-zapovednik-kak-obekt-regionalnogo-i-globalnogo-geosistemnogo-monitoringa-na-primere-prioksko-terrasnogo-zapovednika",
    "https://cyberleninka.ru/article/n/dinamika-zapasov-biofilnyh-elementov-v-valezhe-i-pochve-posle-massovogo-vetrovala-v-shirokolistvennom-lesu-na-flyuvioglyatsialnyh",
    "https://cyberleninka.ru/article/n/gidrofilno-travyanaya-humido-herbetion-rastitelnost-bolot-srednerusskoy-vozvyshennosti",
    "https://cyberleninka.ru/article/n/zalezhnye-zemli-rossii-rasprostranenie-agroekologicheskoe-sostoyanie-i-perspektivy-ispolzovaniya-obzor",
    "https://cyberleninka.ru/article/n/gidrofilno-mohovaya-rastitelnost-bolot-srednerusskoy-vozvyshennosti",
    "https://cyberleninka.ru/article/n/mikrobnaya-biomassa-i-fermentativnaya-aktivnost-lesnyh-pochv-raifskogo-uchastka-volzhsko-kamskogo-zapovednika",
    "https://cyberleninka.ru/article/n/preparaty-na-osnove-bakteriy-i-gumatov-dlya-lesovosstanovleniya-i-povysheniya-deponirovaniya-ugleroda-drevesnymi-rasteniyami",
    "https://cyberleninka.ru/article/n/raznoobrazie-tipov-bolot-tulskoy-oblasti",
    "https://cyberleninka.ru/article/n/bolota-severnoy-evrazii-biosfernye-funktsii-raznoobrazie-i-upravlenie-petrozavodsk-25-28-sentyabrya-2023-g",
    "https://cyberleninka.ru/article/n/puly-i-fraktsii-organicheskogo-ugleroda-v-pochve-struktura-funktsii-i-metody-opredeleniya",
    "https://cyberleninka.ru/article/n/trendy-vosproizvodstva-postagrogennyh-pochv-v-lesostepnoy-zone-evropeyskoy-territorii-rossii",
    "https://cyberleninka.ru/article/n/otsenka-zapasov-ugleroda-v-torfyanyh-zalezhah-razlichnogo-genezisa-na-bolotah-severo-vostoka-srednerusskoy-vozvyshennosti",
    "https://cyberleninka.ru/article/n/vliyanie-senokosheniya-i-vypasa-na-floristicheskiy-sostav-i-zapasy-nadzemnoy-biomassy-soobschestv-lugovoy-stepi-na-zalezhah-v",
    "https://cyberleninka.ru/article/n/monitoring-poverhnostnyh-vod-kuybyshevskogo-vodohranilischa-v-rayone-vodnogo-uchastka-poligona-karbon-povolzhie",
    "https://cyberleninka.ru/article/n/nadzemnaya-i-podzemnaya-fitomassy-lugovyh-soobschestv-poymy-sredney-obi-krivosheinskiy-rayon-tomskoy-oblasti",
    "https://cyberleninka.ru/article/n/osobennosti-suktsessii-rastitelnosti-na-primere-rekultivirovannogo-otvala-ugolnogo-razreza-chernogorskiy",
    "https://cyberleninka.ru/article/n/vzaimosvyaz-funktsionalnyh-priznakov-i-nadzemnoy-biomassy-rasteniy-travyano-kustarnichkovogo-yarusa-v-ekosisteme-karbonovogo",
    "https://cyberleninka.ru/article/n/monitoring-vidovogo-sostava-i-ekologo-tsenoticheskih-harakteristik-sfagnovyh-mhov-na-karbonovom-poligone-rosyanka-kaliningradskaya",
    "https://cyberleninka.ru/article/n/otsenka-zapasov-ugleroda-v-pochve-na-territorii-karbonovogo-poligona-karbon-povolzhie",

    # Сельское хозяйство, лесное хозяйство, рыбное хозяйство (31)
    "https://cyberleninka.ru/article/n/ekologiya-zhizn-v-neustoychivoy-biosfere",
    "https://cyberleninka.ru/article/n/voprosy-lesovodstvennogo-sovershenstvovaniya-sistemy-sohraneniya-i-ispolzovaniya-lesov-v-ramkah-resheniya-problemy-adaptatsii-lesov",
    "https://cyberleninka.ru/article/n/strategiya-sohraneniya-biologicheskogo-raznoobraziya-ohrany-lesnyh-i-vodnyh-resursov-borealnoy-zony",
    "https://cyberleninka.ru/article/n/osobennosti-nakopleniya-fitomassy-v-lesnyh-nasazhdeniyah-na-otvalah-listvyanskogo-ugolnogo-razreza",
    "https://cyberleninka.ru/article/n/otsenka-vklada-rossiyskih-lesov-v-snizhenie-riskov-klimaticheskih-izmeneniy",
    "https://cyberleninka.ru/article/n/voprosy-adaptivno-tselevogo-primeneniya-lesovodstvennyh-rubok-v-tselyah-povysheniya-effektivnosti-deponirovaniya-i-konservatsii",
    "https://cyberleninka.ru/article/n/formirovanie-nadzemnoy-fitomassy-listvennyh-drevesnyh-porod-na-postagrogennyh-zemlyah",
    "https://cyberleninka.ru/article/n/opredelenie-perspektivnosti-sosny-obyknovennoy-pinus-sylvestris-l-dlya-sozdaniya-karbonovyh-ferm",
    "https://cyberleninka.ru/article/n/determinanty-povysheniya-effektivnosti-ispolzovaniya-drevesnyh-lesnyh-resursov",
    "https://cyberleninka.ru/article/n/snizhenie-riskov-klimaticheskih-izmeneniy-i-ih-posledstviy-dlya-ekonomiki-otsenka-vklada-rossiyskih-lesov",
    "https://cyberleninka.ru/article/n/funktsionalno-tselevaya-karbonovaya-sistema-obektov-lesovodstva-uglerododeponiruyuschego-i-uglerodokonservatsionnogo-naznacheniya",
    "https://cyberleninka.ru/article/n/obosnovanie-primeneniya-sorbtsionnogo-materiala-dlya-ochistki-neftezagryaznyonnyh-stokov-predpriyatiy-lesopromyshlennoy",
    "https://cyberleninka.ru/article/n/strategicheskoe-znachenie-diversifikatsii-rastenievodstva",
    "https://cyberleninka.ru/article/n/faktory-territorialnoy-differentsiatsii-agrolandshafta-i-perspektivy-sohraneniya-stepey-belgorodskoy-oblasti",
    "https://cyberleninka.ru/article/n/sravnitelnaya-otsenka-metodov-uchyota-deponirovaniya-ugleroda-sosnovo-beryozovymi-lesnymi-nasazhdeniyami-voronezhskoy-oblasti",
    "https://cyberleninka.ru/article/n/karbonovye-fermy-otvaly-ugolnyh-predpriyatiy-kuzbassa",
    "https://cyberleninka.ru/article/n/optimizatsiya-priemov-osnovnoy-obrabotki-pochvy-na-sklonovyh-landshaftah-chechenskoy-respubliki",
    "https://cyberleninka.ru/article/n/akkumulyatsiya-ugleroda-v-pochve-i-fitomasse-zaschitnyh-lesnyh-nasazhdeniy-yuga-rossii",
    "https://cyberleninka.ru/article/n/otsenka-uglerodsekvestriruyuschego-potentsiala-pochv-agrolandshaftov-srednego-urala",
    "https://cyberleninka.ru/article/n/ekosistemnye-uslugi-zalezhnyh-pochv-leningradskoy-oblasti-rossiyskaya-federatsiya",
    "https://cyberleninka.ru/article/n/ekosistemnye-uslugi-zalezhnyh-pochv-leningradskoy-oblasti-rossiyskaya-federatsiya-1",
    "https://cyberleninka.ru/article/n/renty-karbonovyh-ferm-na-zemlyah-selskohozyaystvennogo-naznacheniya",
    "https://cyberleninka.ru/article/n/bioenergeticheskie-i-reproduktsionnye-nasazhdeniya-nastoyaschih-topoley-v-tsentralnom-chernozemie-rossii",
    "https://cyberleninka.ru/article/n/nauchnye-issledovaniya-i-razrabotki-uralskogo-gosudarstvennogo-lesotehnicheskogo-universiteta-v-oblasti-klimaticheskih-proektov",
    "https://cyberleninka.ru/article/n/issledovanie-metodov-i-obrabotka-baz-dannyh-o-biomasse-lesov-evrazii-kak-neyronnyh-setey-chast-2-novye-vozmozhnosti-iskusstvennogo",
    "https://cyberleninka.ru/article/n/analiz-sovremennogo-sostoyaniya-agrolandshaftov-i-puti-povysheniya-ih-sekvestratsionnogo-potentsiala-pri-selskohozyaystvennom",
    "https://cyberleninka.ru/article/n/poligon-ural-karbon-severka",
    "https://cyberleninka.ru/article/n/monitoring-vybrosov-parnikovyh-gazov-na-karbonovom-poligone-way-carbon-metodom-regenerativnogo-zhivotnovodstva",
    "https://cyberleninka.ru/article/n/karbonovoe-zemledelie-kak-element-klimaticheskoy-strategii",
    "https://cyberleninka.ru/article/n/pilotnyy-karbonovyy-poligon-v-rossii-analiz-zapasov-ugleroda-v-pochvah-i-rastitelnosti",
    "https://cyberleninka.ru/article/n/prirodopodobnaya-agrotehnologiya-konvergentnyy-podhod-k-zakladke-karbonovyh-poligonov",

    # Экономика и бизнес (53)
    "https://cyberleninka.ru/article/n/upravlenie-zelenym-regionom-kontseptsiya-ekologo-orientirovannogo-regionalnogo-razvitiya",
    "https://cyberleninka.ru/article/n/kurit-ili-dyshat-otsenka-vliyaniya-tabachnoy-industrii-na-ekologicheskuyu-situatsiyu",
    "https://cyberleninka.ru/article/n/o-sovershenstvovanii-zakonodatelstva-v-oblasti-razvitiya-innovatsionnyh-proektov-v-neftegazovom-sektore-ekonomiki",
    "https://cyberleninka.ru/article/n/otrazhenie-kontseptsii-energoperehoda-v-strategiyah-razvitiya-otrasley-i-regionov-rossii",
    "https://cyberleninka.ru/article/n/sotsialno-ekonomicheskoe-blagopoluchie-cheloveka-v-usloviyah-vozdeystviya-ekologicheskih-ugroz",
    "https://cyberleninka.ru/article/n/transformatsiya-upravleniya-lesopromyshlennym-kompleksom-baykalskogo-regiona-pri-perehode-na-novyy-tehnologicheskiy-uklad",
    "https://cyberleninka.ru/article/n/predposylki-realizatsii-klimaticheskih-proektov-v-rossii",
    "https://cyberleninka.ru/article/n/adaptatsiya-zemledelcheskogo-proekta-k-regulirovaniyu-vybrosov-parnikovyh-gazov",
    "https://cyberleninka.ru/article/n/dalniy-vostok-rossii-i-severo-vostochnaya-aziya-novye-perspektivy-i-novye-riski",
    "https://cyberleninka.ru/article/n/mify-i-rify-klimaticheskoy-povestki",
    "https://cyberleninka.ru/article/n/energoperehod-podhody-k-formirovaniyu-povestki-issledovaniy-dlya-rossiyskoy-nauki",
    "https://cyberleninka.ru/article/n/osnovnye-aspekty-razrabotki-strategii-nauchnotehnologicheskogo-razvitiya-respubliki-tatarstan",
    "https://cyberleninka.ru/article/n/vnedrenie-innovatsiy-kak-prevratit-administrativnye-bariery-v-administrativnye-trampliny",
    "https://cyberleninka.ru/article/n/obosnovanie-resheniy-v-oblasti-tsifrovizatsii-konteynernyh-servisov-oktyabrskoy-zheleznoy-dorogi",
    "https://cyberleninka.ru/article/n/ekologicheskiy-vektor-ustoychivogo-razvitiya-i-ego-pravovoe-obespechenie",
    "https://cyberleninka.ru/article/n/sokraschenie-azotnogo-pitaniya-rasteniy-dlya-borby-s-parnikovym-effektom-otsenka-posledstviy-dlya-selskogo-hozyaystva",
    "https://cyberleninka.ru/article/n/razvitie-zelyonoy-ekonomiki-brazilii-i-ekonomicheskie-interesy-rossii",
    "https://cyberleninka.ru/article/n/tipologizatsiya-regionalnyh-sistem-rossii-v-svyazi-s-protsessami-dekarbonizatsii-ekonomiki",
    "https://cyberleninka.ru/article/n/analiz-investitsionnoy-deyatelnosti-promyshlennyh-predpriyatiy-rf-v-kontekste-zelenoy-ekonomiki",
    "https://cyberleninka.ru/article/n/otsenka-sotsialno-ekonomicheskogo-polozheniya-i-potentsiala-razvitiya-sibirskogo-federalnogo-okruga",
    "https://cyberleninka.ru/article/n/spetsifika-zelenyh-innovatsiy-i-ih-realizatsii-v-promyshlennosti",
    "https://cyberleninka.ru/article/n/globalnoe-poteplenie-i-energeticheskiy-perehod-vneshneekonomicheskiy-aspekt",
    "https://cyberleninka.ru/article/n/regionalnyy-opyt-podgotovki-kadrov-dlya-sotsialno-ekonomicheskogo-razvitiya-chuvashskoy-respubliki",
    "https://cyberleninka.ru/article/n/instrumenty-obespecheniya-uglerodnoy-neytralnosti-v-rossiyskom-ugolnom-biznese",
    "https://cyberleninka.ru/article/n/matritsa-institutsionalnogo-kapitala-regiona",
    "https://cyberleninka.ru/article/n/osobennosti-funktsionirovaniya-vnegorodskih-territoriy-v-usloviyah-realizatsii-strategii-prostranstvennogo-razvitiya-rossii",
    "https://cyberleninka.ru/article/n/osobennosti-realizatsii-regionalnoy-promyshlennoy-politiki-na-primere-respubliki-bashkortostan",
    "https://cyberleninka.ru/article/n/potentsial-prirodno-ekonomicheskogo-ekosistemnogo-ucheta-dlya-podderzhki-upravleniya-pribrezhnymi-i-morskimi-territoriyami-na",
    "https://cyberleninka.ru/article/n/lesnye-klimaticheskie-proekty-vozmozhnosti-i-problemy-realizatsii-esg-podhoda-chast-2",
    "https://cyberleninka.ru/article/n/globalnaya-ekonomicheskaya-povestka-biznes-strategii-ustoychivogo-razvitiya-esg-i-ee-lokalnye-prioritety-vostrebovannost",
    "https://cyberleninka.ru/article/n/apgreyd-tsepochek-dobavlennoy-stoimosti-rossiyskih-ugolnyh-kompaniy-i-ih-vliyanie-na-stressoustoychivost-otrasli-v-usloviyah",
    "https://cyberleninka.ru/article/n/opredelenie-podhodov-k-vyboru-upravlencheskih-resheniy-v-ekologicheskom-sektore-ekonomiki-predprinimatelstva",
    "https://cyberleninka.ru/article/n/upravlenie-realizatsiey-lesoklimaticheskih-proektov-v-rf-perspektivy-i-riski",
    "https://cyberleninka.ru/article/n/metod-otsenki-investitsionnoy-privlekatelnosti-klimaticheskih-proektov-na-baze-rascheta-uglerodoemkosti-investitsionnyh-zatrat",
    "https://cyberleninka.ru/article/n/osnovnye-instrumenty-gosudarstvennogo-regulirovaniya-transformatsii-toplivno-energeticheskogo-balansa",
    "https://cyberleninka.ru/article/n/formirovanie-agrarnogo-uchebno-nauchno-proizvodstvennogo-klastera-agrotehnopark",
    "https://cyberleninka.ru/article/n/razvitie-zelenoy-ekonomiki-v-lpk-regionov-sibiri",
    "https://cyberleninka.ru/article/n/issledovanie-praktiki-realizatsii-esg-povestki-v-regionah",
    "https://cyberleninka.ru/article/n/regionalnye-aspekty-razvitiya-rossii-v-ramkah-realizatsii-tseley-ustoychivogo-razvitiya",
    "https://cyberleninka.ru/article/n/harakteristika-biznes-protsessa-karbonovoy-fermy",
    "https://cyberleninka.ru/article/n/xi-mezhregionalnaya-agropromyshlennaya-vystavka-uralskogo-federalnogo-okruga-obzor-meropriyatiy-i-kruglyh-stolov",
    "https://cyberleninka.ru/article/n/swot-analiz-razvitiya-regionalnyh-territoriy-vyzovy-ugrozy-i-faktory-vliyayuschie-na-ekonomicheskuyu-bezopasnost",
    "https://cyberleninka.ru/article/n/pedagogicheskie-zadachi-vysshey-shkoly-v-perehode-k-tsirkulyarnoy-modeli-ekonomiki",
    "https://cyberleninka.ru/article/n/karbonovoe-zemleustroystvo",
    "https://cyberleninka.ru/article/n/rol-tretiey-missii-otraslevogo-universiteta-v-razvitii-regionalnoy-ekonomiki",
    "https://cyberleninka.ru/article/n/formirovanie-i-razvitie-rossiyskogo-rynka-uglerodnyh-edinits",
    "https://cyberleninka.ru/article/n/ustoychivoe-razvitie-i-esg-novaya-epoha-upravleniya",
    "https://cyberleninka.ru/article/n/izmenenie-klimata-globalnye-trendy-i-rossiyskaya-povestka",
    "https://cyberleninka.ru/article/n/otsenka-masshtabov-i-perspektiv-vliyaniya-klimaticheskih-riskov-na-sotsialno-ekonomicheskoe-razvitie-rossii",
    "https://cyberleninka.ru/article/n/problemy-dekarbonizatsii-ekonomiki-rossii",
    "https://cyberleninka.ru/article/n/dekarbonizatsiya-po-sibirski",
    "https://cyberleninka.ru/article/n/rol-karbonovoe-zemledelie-v-ekonomicheskoy-stabilnosti-rossii",
    "https://cyberleninka.ru/article/n/razvitie-karbonovyh-poligonov-v-rossii",

    # Науки о Земле и смежные экологические науки (65)
     "https://cyberleninka.ru/article/n/dinamika-ozernosti-volgo-meshinskogo-mezhdurechya",
    "https://cyberleninka.ru/article/n/sanitarno-ekologicheskaya-otsenka-kachestva-vod-vostochnoy-chasti-finskogo-zaliva-po-rezultatam-mikrobiologicheskih-issledovaniy",
    "https://cyberleninka.ru/article/n/zakonomernosti-raspredeleniya-zapasov-ugleroda-v-pochvah-i-lesnoy-podstilke-volzhsko-kamskogo-zapovednika",
    "https://cyberleninka.ru/article/n/vliyanie-razlichnyh-sistem-zemledeliya-na-agrohimicheskie-i-biohimicheskie-pokazateli-seroy-lesnoy-pochvy",
    "https://cyberleninka.ru/article/n/otsenka-vliyaniya-temperatury-vegetatsii-na-effektivnost-ulavlivaniya-atmosfernogo-ugleroda-rasteniyami-viki-yarovoy-vicia-sativa-l",
    "https://cyberleninka.ru/article/n/sravnenie-metodov-segmentatsii-rastrovyh-izobrazheniy-i-ih-neyrosetevaya-klassifikatsiya",
    "https://cyberleninka.ru/article/n/razvitie-mehanizmov-upravleniya-kachestvom-atmosfernogo-vozduha-v-respublike-tatarstan",
    "https://cyberleninka.ru/article/n/sposob-raschyota-kontsentratsiy-dioksida-ugleroda-postupayuschego-v-atmosfernyy-vozduh-goroda-s-vybrosami-promyshlennyh",
    "https://cyberleninka.ru/article/n/vertikalnoe-raspredelenie-dozhdevyh-chervey-v-pochve-sosnyakov-i-berezovo-osinovyh-lesov-na-primere-lesostepnogo-priobya",
    "https://cyberleninka.ru/article/n/terreynovaya-tektonika-tsentralno-aziatskogo-skladchatogo-poyasa",
    "https://cyberleninka.ru/article/n/paleozoyskie-i-mezozoyskie-ochagi-generatsii-uglevodorodov-i-otsenka-ih-roli-v-formirovanii-zalezhey-doyurskogo-kompleksa-zapadnoy",
    "https://cyberleninka.ru/article/n/gumusovaya-melioratsiya-narushennyh-zemel-proshloe-nastoyaschee-buduschee-obzor",
    "https://cyberleninka.ru/article/n/glubinnoe-stroenie-salairskogo-skladchato-pokrovnogo-sooruzheniya-severo-zapad-tsentralno-aziatskogo-skladchatogo-poyasa",
    "https://cyberleninka.ru/article/n/rannevizeyskiy-evksinskiy-basseyn-v-sisteme-paleotetisa-v-svete-sovremennyh-dannyh",
    "https://cyberleninka.ru/article/n/vliyanie-geologicheskih-faktorov-na-raspredelenie-urana-i-toriya-v-solevyh-otlozheniyah-pitievyh-vod-respublika-bashkortostan",
    "https://cyberleninka.ru/article/n/geohimicheskie-bariery-kak-forma-samoorganizatsii-estestvennyh-geosictem",
    "https://cyberleninka.ru/article/n/geohimicheskie-osobennosti-i-ekologicheskoe-sostoyanie-malyh-rek-priangarya-na-primere-rek-bohanskogo-rayona",
    "https://cyberleninka.ru/article/n/razrabotka-ostatochnyh-zapasov-kumakskogo-zolotorudnogo-mestorozhdeniya-orenburgskaya-oblast",
    "https://cyberleninka.ru/article/n/diagnostika-i-mezhregionalnaya-korrelyatsiya-srednepleystotsenovyh-lednikovyh-gorizontov-tsentra-i-severo-vostoka-russkoy-raniny",
    "https://cyberleninka.ru/article/n/evolyutsionnaya-model-noveyshey-struktury-amuro-hankayskogo-riftogena-osnovnye-zakonomernosti-razvitiya-i-rol-golotsenovyh",
    "https://cyberleninka.ru/article/n/kotloviny-srednesibirskogo-ploskogorya-raspolozhenie-morfologiya-i-sovremennoe-reliefoobrazovanie",
    "https://cyberleninka.ru/article/n/fenomen-globalnoy-golotsenovoy-aktivizatsii-na-primerah-noveyshih-struktur-vostoka-azii-srednego-povolzhya-i-namibii-indikatory",
    "https://cyberleninka.ru/article/n/strukturno-funktsionalnye-osobennosti-landshaftov-s-aktivnym-termokarstom-v-severnoy-tayge-zapadnoy-sibiri",
    "https://cyberleninka.ru/article/n/prognoz-neftegazonosnosti-vilyuyskoy-gemisineklizy-na-baze-interpretatsii-geologo-geofizicheskih-materialov-i-tehnologii",
    "https://cyberleninka.ru/article/n/distantsionnyy-monitoring-teplovogo-polya-neftyanogo-mestorozhdeniya-kak-instrument-upravleniya-promyslovymi-protsessami",
    "https://cyberleninka.ru/article/n/evolyutsionnaya-model-noveyshey-struktury-shantarskogo-arhipelaga",
    "https://cyberleninka.ru/article/n/eotsen-golotsenovaya-evolyutsiya-sinegorskoy-rudno-magmaticheskoy-sistemy-yuzhnoe-primorie",
    "https://cyberleninka.ru/article/n/osobennosti-raspredeleniya-i-sostava-snezhnogo-pokrova-v-predelah-landshaftov-na-territorii-uopets-mgu-chashnikovo",
    "https://cyberleninka.ru/article/n/tipologiya-i-rasprostranenie-bolot-na-srednerusskoy-vozvyshennosti",
    "https://cyberleninka.ru/article/n/biohimicheskaya-transformatsiya-tvyordyh-kommunalnyh-othodov-v-pogrebyonnyh-nesanktsionirovannyh-svalkah",
    "https://cyberleninka.ru/article/n/ispolzovanie-pokazateley-otrazhatelnoy-sposobnosti-kak-kovariaty-soderzhaniya-organicheskogo-veschestva-v-zalezhnyh-pochvah",
    "https://cyberleninka.ru/article/n/opasnost-obektov-nakoplennogo-vreda-dlya-okruzhayuschey-sredy",
    "https://cyberleninka.ru/article/n/karta-raspredeleniya-plotnosti-teplovogo-potoka-krovli-fundamenta-vostochnoy-chasti-tomskoy-oblasti",
    "https://cyberleninka.ru/article/n/turistsko-rekreatsionnoe-osvoenie-kulturnyh-landshaftov-makazhoyskoy-kotloviny-potentsial-ugrozy-i-perspektivy",
    "https://cyberleninka.ru/article/n/ispolzovanie-zimnih-i-rannevesennih-kosmosnimkov-dlya-otsenki-proektivnogo-pokrytiya-bereznyakov-na-zalezhah",
    "https://cyberleninka.ru/article/n/proishozhdenie-drevnih-ekzoticheskih-glyb-v-krymu",
    "https://cyberleninka.ru/article/n/ispolzovaniya-podzemnyh-vod-dlya-ohlazhdeniya-superkompyuterov-mgu-na-primere-superkompyutera-lomonosov",
    "https://cyberleninka.ru/article/n/paleogeograficheskie-zakonomernosti-izmenchivosti-sostava-moren-v-tsentralnoy-chasti-vostochno-evropeyskoy-ravniny",
    "https://cyberleninka.ru/article/n/dinamika-ekologicheskih-usloviy-v-genezise-raznyh-tipov-bolot-severnoy-chasti-srednerusskoy-vozvyshennosti",
    "https://cyberleninka.ru/article/n/pervye-rezultaty-izucheniya-prostranstvennovremennoy-izmenchivosti-soderzhaniya-i-emissii-metana-v-tsimlyanskom-vodohranilische",
    "https://cyberleninka.ru/article/n/iskusstvennye-vodoemy-kak-ochagi-metanogeneza-na-primere-tsimlyanskogo-vodohranilischa",
    "https://cyberleninka.ru/article/n/vliyanie-apvellinga-na-raspredelenie-hlorofilla-a-v-pribrezhnoy-zone-yugo-vostochnoy-baltiki-v-letniy-period-2000-2019-godov",
    "https://cyberleninka.ru/article/n/geoekologicheskie-problemy-mestorozhdeniya-termalnyh-vod-gunyushki",
    "https://cyberleninka.ru/article/n/modelirovanie-vertikalnogo-raspredeleniya-soderzhaniya-ugleroda-v-pochvah-pri-raznyh-tipah-zemlepolzovaniya",
    "https://cyberleninka.ru/article/n/emissiya-parnikovyh-gazov-iz-prirodnyh-ekosistem-norilskogo-promyshlennogo-rayona",
    "https://cyberleninka.ru/article/n/osobennosti-mikroelementnogo-sostava-gorno-lugovyh-i-buryh-lesnyh-pochv-vostochnoy-chasti-zapadnogo-kavkaza",
    "https://cyberleninka.ru/article/n/uglerodnyy-status-rossii-i-dinamicheskoe-ravnovesie-biosfery",
    "https://cyberleninka.ru/article/n/vliyanie-klimaticheskih-izmeneniy-na-rasprostranenie-bakterialnoy-vodyanki-berezy",
    "https://cyberleninka.ru/article/n/dinamika-produktivnosti-vosstanovlennoy-rastitelnosti-ugolnogo-razreza-chernogorskiy-po-sputnikovym-dannym-terra-modis",
    "https://cyberleninka.ru/article/n/uglerododeponiruyuschiy-potentsial-embrionalnyh-pochv-na-vskryshnyh-porodah-starooskolskogo-zhelezorudnogo-rayona",
    "https://cyberleninka.ru/article/n/mezhdunarodnaya-konferentsiya-po-voprosam-izmeneniya-klimata-i-tayaniya-vechnoy-merzloty",
    "https://cyberleninka.ru/article/n/otsenka-soderzhaniya-zapasov-ugleroda-v-gorodskih-ekosistemah-na-primere-nizhegorodskoy-aglomeratsii",
    "https://cyberleninka.ru/article/n/otsenka-dinamiki-prostranstvenno-vremennogo-izmeneniya-urovnya-evtrofikatsii-poverhnostnyh-vodoemov",
    "https://cyberleninka.ru/article/n/rol-statsionarnyh-issledovaniy-dinamiki-gornyh-landshaftov-severnogo-kavkaza-na-primere-digorskogo-uschelya",
    "https://cyberleninka.ru/article/n/k-voprosu-ob-izuchenii-dinamiki-ugleroda-v-gornyh-landshaftah-chechenskoy-respubliki-na-osnove-landshaftno-geofizicheskogo-podhoda",
    "https://cyberleninka.ru/article/n/osobennosti-sutochnoy-emissii-metana-v-sisteme-chernozemnye-pochvy-atmosfera-i-vliyanie-vlazhnosti-pochv-na-skorost-ego-potokov",
    "https://cyberleninka.ru/article/n/kompleksnyy-podhod-k-otsenke-izmeneniy-v-zemlepolzovanii-i-ih-vliyaniya-na-dinamiku-ugleroda-v-gornyh-landshaftah-chechenskoy",
    "https://cyberleninka.ru/article/n/nekotorye-vyzovy-i-vozmozhnosti-dlya-rossii-i-regionov-v-plane-globalnogo-trenda-dekarbonizatsii",
    "https://cyberleninka.ru/article/n/modelirovanie-dinamiki-kontsentratsii-rastvorennogo-kisloroda-v-vodah-zaliva-aniva-ohotskoe-more",
    "https://cyberleninka.ru/article/n/kontsentratsiya-i-emissiya-metana-v-razlichnyh-tipah-pochv-rostovskoy-oblasti",
    "https://cyberleninka.ru/article/n/pervye-otsenki-potokov-uglekislogo-gaza-na-territorii-karbonovogo-poligona-gelendzhik",
    "https://cyberleninka.ru/article/n/analiz-zapasov-ugleroda-v-pochvah-na-uchastkah-karbonovogo-poligona-respubliki-bashkortostan",
    "https://cyberleninka.ru/article/n/tsifrovaya-model-karbonovogo-poligona-v-krasnodarskom-krae-na-baze-yuo-io-ran-g-gelendzhik",
    "https://cyberleninka.ru/article/n/regionalnye-landshaftnye-osobennosti-sozdaniya-karbonovogo-poligona-v-respublike-dagestan",
    "https://cyberleninka.ru/article/n/karbonovyy-poligon-chechenskoy-respubliki-iii-geofizicheskie-issledovaniya-na-karbonovoy-ferme"
]

# Создание CSV файла с одним столбцом 'Link'
with open('links.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for link in links:
        writer.writerow({'Link': link})
